In [ ]:
#USER INPUTS BElOW

#Number of splines to be constructed
n_splines = 10
#Number of pixels per axis
division =15
#Noise for grey image construction
noise = 5

#Creating splines by solving a cubic differential equation in parametric form
import numpy as np
import pandas as pd
import plotly.graph_objects as go


#Extrapolating from a given point in one direction (20 points)
def hermite_0(f,df,d2f,d3f):
    a = d3f/6
    b = d2f/2
    c = df
    d = f
    t = np.linspace(0,1,20)
    F = a*t**3+b*t**2+c*t+d
    F1 = a+b+c+d
    DF1 = 3*a+2*b+c
    D2F1 = 6*a+2*b
    D3F1 = 6*a
    result = list([F,F1,DF1,D2F1])
    return result

#Extrapolating from a given point in the other direction (20 points)
def hermite_1(f,df,d2f,d3f):
    a = d3f/6
    b = (d2f-d3f)/2
    c = df-d2f+d3f/2
    d = f-df+d2f/2-d3f/6
    t = np.linspace(0,1,20)
    F = a*t**3+b*t**2+c*t+d
    F0 = d
    DF0 = c
    D2F0 = 2*b
    D3F0 = 6*a
    result = list([F,F0,DF0,D2F0])
    return result


#Does not let any two points from any two splines come closer than a user defined distance
def rejection_sampling(s1, s2):
    decision = 1
    for i in range(len(s1)):
        for j in range(len(s2)):
            dist = np.linalg.norm(s1[i]-s2[j])
            if dist< 0.004:
                decision  = 0
                break

    return decision



splines = []
r=0

#Construct splines starting from a random point and extrapolating until it reaches the boundaries of the unit cube
for i in range(n_splines):
    
    x_i = np.random.uniform(0,10)
    dfx_i = np.random.uniform(-10,10)
    d2fx_i = np.random.uniform(-10,10)
    
    y_i = np.random.uniform(0,10)
    dfy_i = np.random.uniform(-10,10)
    d2fy_i = np.random.uniform(-10,10)
    
    z_i = np.random.uniform(0,10)
    dfz_i = np.random.uniform(-10,10)
    d2fz_i = np.random.uniform(-10,10)

    terminate = False
    Y_r = []
    Z_r = []
    X_r = []
    x_r = hermite_0(x_i, dfx_i, d2fx_i, np.random.uniform(-r,r))
    y_r = hermite_0(y_i, dfy_i, d2fy_i, np.random.uniform(-r,r))
    z_r = hermite_0(z_i, dfz_i, d2fz_i, np.random.uniform(-r,r))
    while terminate == False:

        Y_r.extend(y_r[0])
        X_r.extend(x_r[0])
        Z_r.extend(z_r[0])
        for cordinate in np.column_stack((x_r[0],y_r[0],z_r[0])):
            if cordinate[0]>10 or cordinate[1]>10 or cordinate[2]>10 or cordinate[0]<0 or cordinate[1]<0 or cordinate[2]<0 :
                terminate = True
                break

        x_r = hermite_0(x_r[1],x_r[2],x_r[3], np.random.uniform(-r,r))
        y_r = hermite_0(y_r[1],y_r[2],y_r[3], np.random.uniform(-r,r))
        z_r = hermite_0(z_r[1],z_r[2],z_r[3], np.random.uniform(-r,r))


    terminate = False
    Y_l = []
    Z_l = []
    X_l = []
    x_l = hermite_1(x_i, dfx_i, d2fx_i, np.random.uniform(-r,r))
    y_l = hermite_1(y_i, dfy_i, d2fy_i, np.random.uniform(-r,r))
    z_l = hermite_1(z_i, dfz_i, d2fz_i, np.random.uniform(-r,r))
    while terminate == False:

        Y_l.extend(np.flip(y_l[0]))
        X_l.extend(np.flip(x_l[0]))
        Z_l.extend(np.flip(z_l[0]))
        for cordinate in np.column_stack((x_l[0],y_l[0],z_l[0])):
            if cordinate[0]>10 or cordinate[1]>10 or cordinate[2]>10 or cordinate[0]<0 or cordinate[1]<0 or cordinate[2]<0 :
                terminate = True
                break

        x_l = hermite_1(x_l[1],x_l[2],x_l[3], np.random.uniform(-r,r))
        y_l = hermite_1(y_l[1],y_l[2],y_l[3], np.random.uniform(-r,r))
        z_l = hermite_1(z_l[1],z_l[2],z_l[3], np.random.uniform(-r,r))

    X_l.reverse()
    Y_l.reverse()
    Z_l.reverse()

    X=np.concatenate((X_l,X_r))
    Y=np.concatenate((Y_l,Y_r))
    Z=np.concatenate((Z_l,Z_r))

    X =X/10
    Y =Y/10
    Z =Z/10



    temp_spline = np.column_stack((X,Y,Z))
    entscheidung = 1
    for x in range(len(splines)):
        entscheidung = entscheidung*rejection_sampling(splines[x], temp_spline)
    if entscheidung == 1:
        splines.append(temp_spline)
#         fig.add_trace(go.Scatter3d(x=X,y=Y,z=Z, marker = dict(size=2)))


'''Creating new_splines to remove the points on the splines that fall outside the unit cube; 
so that out ground truth only has points inside the unit cube'''

b = []
splines = np.array(splines, dtype =object)
for spline in splines:
    b = ((spline>=0)*(spline<=1))
    b = np.array(b)
    spline[~b] =np.nan
    
new_splines = []
for spline in splines:
    spline = spline[~pd.isna(spline).any(axis=1)]
    new_splines.append(spline)

#Plotting the new_splines after removing the points outside the unit cube

fig = go.Figure()
for spline in new_splines:
    fig.add_trace(go.Scatter3d(x=np.array(spline)[:,0],y=np.array(spline)[:,1],z=np.array(spline)[:,2], marker = dict(size=2), line = dict(width =2))
)
fig.update_layout(
    scene = dict(
        xaxis = dict( range=[0,1],),
        yaxis = dict( range=[0,1],),
        zaxis = dict( range=[0,1],)))
fig.show()
# fig.write_image("fig1.pdf")

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig1.pdf")

#Creating a pixel space
import numpy
import random

x = numpy.linspace(0, 1, division)
y = numpy.linspace(0, 1, division)
z = numpy.linspace(0, 1, division)
mesh = numpy.meshgrid(x, y, z)
pixels = list(zip(*(dim.flat for dim in mesh)))


#Indexing pixels (ipixels); so that each pixel can be accessed with x,y, and z co-ordinates
ipixels = np.array(pixels)
ipixels = ipixels.reshape(division**2,division*3)
ipixels = np.transpose(ipixels)
ipixels = ipixels.reshape(division*3, division,division)
ipixels = np.transpose(ipixels)
ipixels = ipixels.reshape(division,division,division,3)

import math

#Finds the distance between any two points
def distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 + (p2[2] - p1[2])**2)


#Finds the closest point to the passed point 'pt' from the group of points 'others'
def closest(pt, others):
    return min(others, key = lambda i: distance(pt, i))


#Mapping each point on each spline to the closest pixel in the mesh; This is the ground truth as each point is now a pixel
from collections import OrderedDict

splines_New = []
splines_New_index = []
for spline in new_splines:
    new_spline_index = []
    new_spline = []
    for point in spline:
        new_point = closest(point,pixels)
        index_pos = np.where((ipixels[:,:,:,0]==new_point[0]) & (ipixels[:,:,:,1]==new_point[1]) & (ipixels[:,:,:,2]==new_point[2]))
        new_point_index = tuple([index_pos[0][0],index_pos[1][0],index_pos[2][0]])
        new_spline_index.append(new_point_index)
        new_spline.append(new_point)
        
    splines_New.append(list(OrderedDict.fromkeys(new_spline)))
    splines_New_index.append(list(OrderedDict.fromkeys(new_spline_index)))
                

#Plotting splines_New (pixels)

fig = go.Figure()
for new_spline in splines_New:
    fig.add_trace(go.Scatter3d(x=np.array(new_spline)[:,0],y=np.array(new_spline)[:,1],z=np.array(new_spline)[:,2], marker = dict(size=2), line = dict(width =2)))

fig.update_layout(
    scene = dict(
        xaxis = dict( range=[0,1],),
        yaxis = dict( range=[0,1],),
        zaxis = dict( range=[0,1],)))
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig3.pdf")

'''Imaging the splines by assigning intensities to pixels based on their closeness to the a spline point;
Inensities for background and foreground are sampled from two normal distributions and the choice of the normal dist. is 
made based on probabilities defined by closeness to a spline'''

grey_pixels = []
theta_1 = -55
theta_2 = 1100

for pixel in pixels:
    sdist_list =[]
    
    for spline in splines:
        s_dist = 1.73
        
        for node in spline:
            dist = np.linalg.norm(pixel-node)
            if dist < s_dist:
                s_dist = dist
        sdist_list.append(s_dist)

    d = min(sdist_list)
    p = 1/(1+np.exp(-1*(theta_1+theta_2*d)))
    N_c = np.abs(np.random.normal(0,noise))
    N_b = 255 - np.abs(np.random.normal(0,noise))
    Ndist_list = [N_c,N_b]   
    
    grey_pixels.append(list(pixel))
    
    grey_value = random.choices(Ndist_list, weights=(1-p,p), k=1)
    
    grey_pixels[-1].append(grey_value[0])

    
# Xp, Yp, Zp, grey_pixels = map(np.array, [Xp, Yp, Zp, grey_pixels])
# grey_pixels = grey_pixels.flatten()

#Volume image based on assigned pixel intensity
grey_pixels = np.array(grey_pixels)

fig = go.Figure(data=go.Volume(
    x=grey_pixels[:,0],
    y=grey_pixels[:,1],
    z=grey_pixels[:,2],
    value=grey_pixels[:,3],
    isomin=0,
    isomax=255,
    opacity=0.05, # needs to be small to see through all surfaces
    surface_count=25, # needs to be a large number for good volume rendering
    ))
fig.update_layout(
    scene = dict(
        xaxis = dict( range=[0,1],),
        yaxis = dict( range=[0,1],),
        zaxis = dict( range=[0,1],)))
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig4_low.pdf")

grey_pixels = np.array(grey_pixels)
grey_pixels.shape

#Indexing grey_pixels (igrey_pixels); so that each pixel can be accessed with x,y, and z co-ordinates
igrey_pixels = grey_pixels.reshape(division, division, division, 4)
igrey_pixels = igrey_pixels.reshape(division**2,division*4)
igrey_pixels = np.transpose(igrey_pixels)
igrey_pixels = igrey_pixels.reshape(division*4, division,division)
igrey_pixels = np.transpose(igrey_pixels)
igrey_pixels = igrey_pixels.reshape(division,division,division,4)

igrey_pixels.shape

#Finding neigbours without returning the actual point - just 6 neighbours
def find_Neighbours(i,j,k):
    I = []
    J = []
    K = []
    neighbours = []
    
    for p in [i-1,i+1]:
        if p >= 0 and p < division:
            I.append(p)
    for p in [j-1,j+1]:
        if p >= 0 and p < division:
            J.append(p)
    for p in [k-1,k+1]:
        if p >= 0 and p < division:
            K.append(p)
    for m in range(len(I)):
        neighbours.append((I[m],j,k))
    for m in range(len(J)):
        neighbours.append((i,J[m],k))
    for m in range(len(K)):
        neighbours.append((i,j,K[m]))
        
#     neighbours.append((i,j,k))
    return neighbours

#Contructing the totpixels without indexing but with co-ordinates and not with actual values like 'pixels'
x = np.arange(0, division)
y = np.arange(0, division)
z = np.arange(0, division)
mesh = np.meshgrid(x, y, z)
totpixels = list(zip(*(dim.flat for dim in mesh)))



#### Seeded Region Growing algorithm

In [ ]:
#Finds regions along with the membrane from the igrey_pixels i.e., after imaging 
import random 

All_regions = []
totpixels_copy = totpixels.copy()

while len(totpixels_copy) > 0:
    junk = []
    region = [random.choice(totpixels_copy)]
    subregion = region
    while len(subregion) != 0:
        summ = 0
        for x in range(len(region)):
            summ = summ + igrey_pixels[region[x]][3]
        mean = summ/len(region)
        n=0
        for pixel in subregion:

            Neighbours  = find_Neighbours(pixel[0],pixel[1],pixel[2])  
            Neighbours = list(set(Neighbours).difference(junk))
            junk.extend(Neighbours)
            junk.append(pixel)
            junk = list(set(junk))
            for neighbour in Neighbours:
                if np.abs(igrey_pixels[neighbour][3] - mean) < 20:
                    region.append(neighbour)
                    n = n+1

        if n==0:
            subregion = []
        else:
            subregion = region[-n:]

    All_regions.append(region)
    totpixels_copy = list(set(totpixels_copy).difference(region))

len(All_regions)

def FindMaxLength(lst):
    maxList = max(lst, key = len)
    lst.remove(maxList)
  
    return lst

#Plotting the output of the region growing algorithm
ohne_background = FindMaxLength(All_regions.copy())
fig = go.Figure()

for i in range(0,len(ohne_background)):
    segplot = []
    for reg in ohne_background[i]:
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1],z=segplot[:,2], marker = dict(size=2), mode='markers'))

#fig.update_layout(width =1000, height =1000)

fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig5.pdf")

#Cross sectional slice

cross_section = [x for x in totpixels if x[0]== int(division/2)]
temp_cross = []
for i in range(0,len(All_regions)):
    temp_cross.append(list(set(cross_section).intersection(All_regions[i])))
cross = [x for x in temp_cross if x != []]


fig = go.Figure()

for i in range(0,len(cross)):
    segplot = []
    for reg in cross[i]:
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1], z=segplot[:,2], marker = dict(size=2), mode='markers'))
#fig.update_layout(width =1000, height =1000)
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig6a.pdf")

#Cross sectional slice

cross_section = [x for x in totpixels if x[1]== int(division/2)]
temp_cross = []
for i in range(0,len(All_regions)):
    temp_cross.append(list(set(cross_section).intersection(All_regions[i])))
cross = [x for x in temp_cross if x != []]


fig = go.Figure()

for i in range(0,len(cross)):
    segplot = []
    for reg in cross[i]:
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1], z=segplot[:,2], marker = dict(size=2), mode='markers'))
#fig.update_layout(width =1000, height =1000)
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig6b.pdf")

#Cross sectional slice

cross_section = [x for x in totpixels if x[2]== int(division/2)]
temp_cross = []
for i in range(0,len(All_regions)):
    temp_cross.append(list(set(cross_section).intersection(All_regions[i])))
cross = [x for x in temp_cross if x != []]


fig = go.Figure()

for i in range(0,len(cross)):
    segplot = []
    for reg in cross[i]:
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1], z=segplot[:,2], marker = dict(size=2), mode='markers'))
#fig.update_layout(width =1000, height =1000)
fig.show()

# fig.write_image("C:/Users/tomri/Desktop/CMS-PROJ mlcv/template/img/ff/fig6c.pdf")

#Getting all pairs of pixels from the total pixel space
import itertools
Pairs = set()

for subset in itertools.combinations(totpixels, 2):
    Pairs.add(frozenset(subset))
    

#Getting all the splines and the background as separate regions and storing them to Truths
temp_copy = totpixels.copy()
all_splines = []
for sp in splines_New_index:
    all_splines += sp

background = list(set(temp_copy)-set(all_splines))
Truths = splines_New_index.copy() + [background]

# #Calculating Rand's Index
# TP = TN = FN = FP = 0

# for pair in Pairs:
    
#     org = 0 
#     for truth in Truths:
#         org  = org + set(pair).issubset(truth)
        
#     seg = 0 
#     for allregion in All_regions:
#         seg  = seg + set(pair).issubset(allregion)
        
#     if org == 1 and seg == 1:
#         TP = TP + 1
        
#     elif org == 0 and seg == 0:
#         TN = TN + 1
        
#     elif org == 1 and seg == 0:
#         FN = FN + 1
    
#     elif org == 0 and seg == 1:
#         FP = FP + 1
        
# R_index = (TP+TN)/(TP+TN+FN+FP)   

#Alternate Rand's index calculation for computation time optimization

X_pairs = set()
Y_pairs = set()

for spline_bck in Truths:

    dopplets = set()

    for dopplet in itertools.combinations(spline_bck, 2):
        dopplets.add(frozenset(dopplet))

    X_pairs.update(dopplets)

for allregion in All_regions:

    dopplets = set()

    for dopplet in itertools.combinations(allregion, 2):
        dopplets.add(frozenset(dopplet))

    Y_pairs.update(dopplets)
    

TP = len(X_pairs.intersection(Y_pairs))
TN = len(Pairs) - len(X_pairs.union(Y_pairs))
FP = len(X_pairs.difference(Y_pairs))
FN = len(Y_pairs.difference(X_pairs))

R_index = (TP+TN)/(TP+TN+FN+FP) 

print("Rand Index: ",R_index)

#Calculating Variation of Information
n =division**3
VI = 0
p = []
q = []
r = np.array([[None]*len(All_regions)]*len(Truths))

for i in range(len(Truths)):
    p.append(len(Truths[i])/n)
    
for j in range(len(All_regions)):
    q.append(len(All_regions[j])/n)

for i in range(len(Truths)):
    for j in range(len(All_regions)):
        r[i][j] = len(set(Truths[i]).intersection(set(All_regions[j])))/n
        
        
for i in range(len(Truths)):
    for j in range(len(All_regions)):
        if r[i][j]!=0:
            VI = VI + r[i][j]*(np.log(r[i][j]/p[i]) + np.log(r[i][j]/q[j]))

VI = -1*VI
        

print("Variation of Information: ",VI)

#### Greedy Additive Edge Contraction Algorithm

In [ ]:

'''Defining costs for edges based on the pixel intensity difference between the two pixels contributing to the edge.
Negative costs for edges having high difference as negative edges should be cut. And we are looking to minimize the 
function'''

cost_Edash = []
cost_E = []
Neigh_pairs = []

for pair in Pairs:
    chi = np.abs(igrey_pixels[pair[0]][3]-igrey_pixels[pair[1]][3])
    C = np.abs([a - b for a, b in zip(pair[0], pair[1])]).sum()
    if chi>20:
        chi = -1*chi
        
    temp = list([set([pair[0]]),set([pair[1]])])
    temp.append(chi)
    
    C = np.abs([a - b for a, b in zip(pair[0], pair[1])]).sum()
    if C==1:
        Neigh_pairs.append(pair)
        cost_E.append(temp)
    
    cost_Edash.append(temp)
    

#Finding the current maximum cost
def Maximum(costs):
    large = -2147483648
    for cost in costs:
        if cost[-1]>large:
            large = cost[-1]
            edge = cost
    return edge, large

#During edge contraction adding the costs of duplicate edges with different costs
def same_edge_sum(testing):    
    final = []
    while len(testing)!=0:

        item1 = testing.pop(0)
        for item2 in testing:
            if set(frozenset(i) for i in item1[:2]) == set(frozenset(i) for i in item2[:2]):
                
                item1[2] = item1[2]+item2[2]
                testing.remove(item2)
                
        final.append(item1)

#     final.extend(testing)
    return final

#Contract the edge and return the unchanged and changed edges separately
def edge_contract(costs, edge):
    
#     costs.remove(edge)
    costs = [value for value in costs if value[:2] != edge[:2]]
    indices = []
    
    r = edge[0].union(edge[1])
    for cost in costs:
        
        if edge[0] in cost:
            indices.append(costs.index(cost))
            cost[cost.index(edge[0])] = r
            
        if edge[1] in cost:
            indices.append(costs.index(cost))
            cost[cost.index(edge[1])] = r
        
        
    q =[costs[i] for i in indices]
    
    costs = [x for x in costs if x not in q]
    
    edge_addition = same_edge_sum(q)
#     costs.extend(edge_addition)

    
    return costs, edge_addition
    

#GAEC Algorithm

while len(cost_E)!=0:
    
    ab, X_ab = Maximum(cost_E)
    if X_ab < 0:
        break
        
    cost_E, cost_E_edges = edge_contract(cost_E, ab)
    cost_Edash, cost_Edash_edges = edge_contract(cost_Edash, ab)
    
#Adjusting cost_E according to the updated costs from cost_Edash
    for item1 in cost_E_edges:
        for item2 in cost_Edash_edges:
            if set(frozenset(i) for i in item1[:2]) == set(frozenset(i) for i in item2[:2]):
                item1[2] = item2[2]
                
    
    cost_E.extend(cost_E_edges)
    cost_Edash.extend(cost_Edash_edges)
            

#Unraveling the sets to separate regions in order to plot later
decomposed = []
for cost in cost_E:
    decomposed.extend(cost[:2])
Decom = list(set(frozenset(item) for item in decomposed)) 

Decom.remove(Decom[5])

#Plotting the decomposed regions
fig = go.Figure()

for set_ in Decom:
    segplot = []
    for reg in list(set_):
        segplot.append(igrey_pixels[reg][:3])
    segplot = np.array(segplot)


    fig.add_trace(go.Scatter3d(x=segplot[:,0],y=segplot[:,1],z=segplot[:,2], marker = dict(size=2), mode='markers'))

fig.show()

#Calculating Rand's Index
TP = TN = FN = FP = 0

for pair in Pairs:
    
    org = 0 
    for truth in Truths:
        org  = org + set(pair).issubset(truth)
        
    seg = 0 
    for allregion in Decom:
        seg  = seg + set(pair).issubset(allregion)
        
    if org == 1 and seg == 1:
        TP = TP + 1
        
    elif org == 0 and seg == 0:
        TN = TN + 1
        
    elif org == 1 and seg == 0:
        FN = FN + 1
    
    elif org == 0 and seg == 1:
        FP = FP + 1
        
R_index = (TP+TN)/(TP+TN+FN+FP)   

print("Rand Index: ",R_index)

#Calculating Variation of Information
n =division**3
VI = 0
p = []
q = []
r = np.array([[None]*len(Decom)]*len(Truths))

for i in range(len(Truths)):
    p.append(len(Truths[i])/n)
    
for j in range(len(Decom)):
    q.append(len(Decom[j])/n)

for i in range(len(Truths)):
    for j in range(len(Decom)):
        r[i][j] = len(set(Truths[i]).intersection(set(Decom[j])))/n
        
        
for i in range(len(Truths)):
    for j in range(len(Decom)):
        if r[i][j]!=0:
            VI = VI + r[i][j]*(np.log(r[i][j]/p[i]) + np.log(r[i][j]/q[j]))

VI = -1*VI
        

print("Variation of Information: ",VI)